### setup

In [1]:
import chromadb
import pandas as pd
# import chromadb.utils.embedding_functions as embedding_functions

/Users/ioeddk/Desktop/auto-score/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/var/folders/p4/c2zzq7zj0dd4pt2mmcjyq7sr0000gn/T/ipykernel_44961/991014038.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# huggingface_ef = embedding_functions.HuggingFaceEmbeddingFunction(
#     api_key="",
#     model_name="google-bert/bert-base-uncased"
# )

In [2]:
try:
    client = chromadb.PersistentClient(path="./database")
    # collection = client.create_collection("BST_question", embedding_function=huggingface_ef)
    collection = client.create_collection(
        name="Queue_question",
        metadata={"hnsw:space": "cosine"} # l2 is the default
    )
except:
    client = chromadb.PersistentClient(path="./database")
    collection = client.get_collection("Queue_question")

# collection = client.create_collection("BST_question")

In [3]:
df = pd.read_csv('data/data_queue_final.csv').iloc[:, 1:]
df.head(2)

,question,student_answer,correct
0,What is the role of a prototype program in pro...,High risk problems are address in the prototyp...,0
1,What is the role of a prototype program in pro...,To simulate portions of the desired final prod...,0


In [5]:
df.shape

(2297, 3)

In [6]:
df['question'].value_counts()

question
What is a queue?                                                                        79
What is a pointer?                                                                      54
What are the similarities between iteration and recursion?                              31
When defining a recursive function, what are possible causes for infinite recursion?    31
How are overloaded functions differentiated by the compiler?                            31
                                                                                        ..
What is a binary search tree?                                                           24
What is a binary tree?                                                                  24
What is a leaf?                                                                         24
What is the height of a tree?                                                           24
What is a tree?                                                                  

In [7]:
df['correct'].value_counts()

correct
0    2240
1      57
Name: count, dtype: int64

In [8]:
df_correct = df.loc[df['correct'] == 1]
df_incorrect = df.loc[df['correct'] == 0]

In [9]:
df_correct['correct'].value_counts()

correct
1    57
Name: count, dtype: int64

In [10]:
df_incorrect['correct'].value_counts()

correct
0    2240
Name: count, dtype: int64

### add response to collection

In [11]:
# add correct responses
l = df_correct['student_answer'].tolist()
ids = [f"id{i}"for i in range(len(l))]
collection.add(
    documents=df_correct['student_answer'].tolist(),
    metadatas=[{"correct": "True"} for _ in range(len(l))],
    ids=ids,
)

2024-02-18 01:47:30.520654 [W:onnxruntime:, helper.cc:67 IsInputSupported] CoreML does not support input dim > 16384. Input:embeddings.word_embeddings.weight, shape: {30522,384}
2024-02-18 01:47:30.521148 [W:onnxruntime:, coreml_execution_provider.cc:81 GetCapability] CoreMLExecutionProvider::GetCapability, number of partitions supported by CoreML: 49 number of nodes in the graph: 323 number of nodes supported by CoreML: 231


In [12]:
# add incorrect responses
l2 = df_incorrect['student_answer'].tolist()

ids = [f"id{i}"for i in range(len(l), len(l2)+len(l))]
collection.add(
    documents=df_incorrect['student_answer'].tolist(),
    metadatas=[{"correct": "False"} for _ in range(len(l2))],
    ids=ids,
)

### test k-nn prediction

In [5]:
q = collection.query(
    query_texts=["A queue is a data structure that follows FIFO principle, meaning that the first element added to the queue will be the first one to be removed."],
    n_results=3
)
correct_sum = [j['correct'] for j in q['metadatas'][0]].count('True')
weighted_vote = (correct_sum/3) >= 0.5
weighted_vote

True

### test queries - correct

In [6]:
results_c = collection.query(
    query_texts=["A queue is a data structure that follows FIFO principle, meaning that the first element added to the queue will be the first one to be removed."],
    n_results=3
)
results_c

{'ids': [['id2', 'id48', 'id2166']],
 'distances': [[0.07562130689620972,
   0.07846885919570923,
   0.07950890064239502]],
 'metadatas': [[{'correct': 'True'},
   {'correct': 'True'},
   {'correct': 'False'}]],
 'embeddings': None,
 'documents': [['A queue stores a set of elements in a particular order.  Its principle of operation is FIFO(first in first out), which means the first element inserted is the first one to be removed.',
   'A queue is a data structure that holds a set of objects, which has a FIFO (first in first out) priority.',
   'A queue is a data type that operates under a FILO (First In Last Out) method. This means that the first element inserted into the queue is the first element removed.']],
 'uris': None,
 'data': None}

In [7]:
collection.query(
    query_texts=["Essentially a array of sorts with a specific order of removal: the first element to be added is the first to be removed."],
    n_results=3
)

{'ids': [['id1335', 'id15', 'id56']],
 'distances': [[0.29995614290237427, 0.3039930462837219, 0.30841559171676636]],
 'metadatas': [[{'correct': 'False'},
   {'correct': 'True'},
   {'correct': 'True'}]],
 'embeddings': None,
 'documents': [['is a list of element where the first one to be removed is the last one inserted',
   'object that stores elements in order that follows first in first out.  the first element added is the first one to come off.',
   'A data structure that can store elements, which has the property that the last item added will be the last to be removed (or first-in-first-out).']],
 'uris': None,
 'data': None}

### test queries - incorrect

In [8]:
# test give up
collection.query(
    query_texts=["I'm not sure"],
    n_results=3
)

{'ids': [['id2291', 'id2292', 'id2219']],
 'distances': [[0.26199022730801724, 0.52954316443988, 0.7310344135908964]],
 'metadatas': [[{'correct': 'False'},
   {'correct': 'False'},
   {'correct': 'False'}]],
 'embeddings': None,
 'documents': [['I’m not very sure? ',
   "Sorry, I don't know. But please give credit. ",
   'not answered']],
 'uris': None,
 'data': None}

In [9]:
# test give up
collection.query(
    query_texts=["I have no idea"],
    n_results=3
)

{'ids': [['id2291', 'id256', 'id2292']],
 'distances': [[0.42656926779575577, 0.5651817321777344, 0.5688415330095503]],
 'metadatas': [[{'correct': 'False'},
   {'correct': 'False'},
   {'correct': 'False'}]],
 'embeddings': None,
 'documents': [['I’m not very sure? ',
   'NO ANSWER',
   "Sorry, I don't know. But please give credit. "]],
 'uris': None,
 'data': None}

In [10]:
# test a legitimate attempt but incorrect
collection.query(
    query_texts=["I think it's like a type of array which uses LIFO?"],
    n_results=3
)

{'ids': [['id1319', 'id1148', 'id746']],
 'distances': [[0.35854285955429077, 0.4374585747718811, 0.458005428314209]],
 'metadatas': [[{'correct': 'False'},
   {'correct': 'False'},
   {'correct': 'False'}]],
 'embeddings': None,
 'documents': [['A data structure that stores data using LIFO.',
   'An array based list that uses an implicit ordering scheme, often using pointers.',
   'array  it is the collection of similar data types ex:int a[10]  ten indicates the size of array. [ ] is index of array, we can give only integer values to array of a.  where as string mean collection of group of characters.  string declarations have a datatype usually causes storage to be allocated in memory that is capable of holding some predetermined number of symbols.    However  Arrays can be declared to contain values of any non reference data type. Multiple arrarys of the same type ']],
 'uris': None,
 'data': None}

In [11]:
# related to the domain but very clearly not a legitimate attempt
collection.query(
    query_texts=["it is a binary tree with black and red nodes"],
    n_results=3
)

{'ids': [['id1703', 'id1700', 'id1692']],
 'distances': [[0.26988691091537476, 0.31101804971694946, 0.3286328911781311]],
 'metadatas': [[{'correct': 'False'},
   {'correct': 'False'},
   {'correct': 'False'}]],
 'embeddings': None,
 'documents': [['it is a binary tree where each node has a unique key, the left child of a node has only values less than that node, and the right child of each node has higher values than that node.',
   'a binary tree with a special organization of data. where the left child of the element is less than it, and the right child is larger than it.',
   'a binary tree in which the data is in order from left to right.']],
 'uris': None,
 'data': None}

In [12]:
# test complete random gibberish
collection.query(
    query_texts=['kangaroo banana hospital'],
    n_results=3
)

{'ids': [['id2202', 'id2281', 'id2288']],
 'distances': [[0.7833630278592145, 0.7900617659617482, 0.7927077748042781]],
 'metadatas': [[{'correct': 'False'},
   {'correct': 'False'},
   {'correct': 'False'}]],
 'embeddings': None,
 'documents': [['it is where you visit the parent then you vistit the children',
   'A queue is a common surgical tool used in most hospitals.',
   'A queue is a bouncy ball where the snowman likes to smile and participate in the lightsaber battle at TreeHacks.']],
 'uris': None,
 'data': None}

In [13]:
# test sentence that is grammatically correct but not relevant
collection.query(
    query_texts=['Have you visited your parents recently?'],
    n_results=3
)

{'ids': [['id2202', 'id2290', 'id2208']],
 'distances': [[0.567520982471511, 0.69892195682147, 0.7312226700170755]],
 'metadatas': [[{'correct': 'False'},
   {'correct': 'False'},
   {'correct': 'False'}]],
 'embeddings': None,
 'documents': [['it is where you visit the parent then you vistit the children',
   'it is ur mum',
   'go to the bottom of the left sub tree and visit the parent and then its children']],
 'uris': None,
 'data': None}

### test queries - maybe's

In [14]:
collection.query(
    query_texts=["a queue is a list of objects in a particular order that is read one at a time starting at the first followed by the second and so on."],
    n_results=3
)

{'ids': [['id1475', 'id5', 'id35']],
 'distances': [[-2.384185791015625e-07,
   0.11749464273452759,
   0.12926989793777466]],
 'metadatas': [[{'correct': 'False'},
   {'correct': 'True'},
   {'correct': 'True'}]],
 'embeddings': None,
 'documents': [['a queue is a list of objects in a particular order that is read one at a time starting at the first followed by the second and so on.',
   'a queue is a data structure that stores elements in a First in First out order.',
   'A queue is a First in First out data structure much like a line for a movie theatre.  The first object in line is the first object to be handled or enacted upon.']],
 'uris': None,
 'data': None}

In [15]:
collection.query(
    query_texts=["a queue either uses FIFO or LIFO, I'm not sure"],
    n_results=3
)

{'ids': [['id10', 'id2287', 'id33']],
 'distances': [[0.16509878635406494, 0.1709906807315008, 0.18880397081375122]],
 'metadatas': [[{'correct': 'True'},
   {'correct': 'False'},
   {'correct': 'True'}]],
 'embeddings': None,
 'documents': [['A queue is a FIFO data structure.',
   'A queue is a LIFO data structure.',
   'A queue is like a stack except it follows FIFO (First in first out) .']],
 'uris': None,
 'data': None}

In [23]:
# import chromadb.utils.embedding_functions as embedding_functions
# huggingface_ef = embedding_functions.HuggingFaceEmbeddingFunction(
#     api_key="",
#     model_name="google-bert/bert-base-uncased"
# )